<a href="https://colab.research.google.com/github/junealexis13/mutation_mdVar_repo/blob/main/rerun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title IMPORT DEPENDENCIES { display-mode: "form" }
%%capture
!pip install MDAnalysis
!pip install plotly
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import os, glob, shutil, time, fnmatch
from MDAnalysis.auxiliary.XVG import XVGReader
import MDAnalysis as mda
from MDAnalysis.analysis import align
import plotly.express as px
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [ ]:
#@title MOUNT DRIVE { display-mode: "form" }
#MOUNT GOOGLE DRIVE and FORCE CONNECTION WITH GDRIVE
drive.mount('/content/drive')

In [ ]:
#@title FOURCE UNMOUNT DRIVE { display-mode: "form" }

#UNMOUNT GOOGLE DRIVE and FORCE DISCONNECTION WITH GDRIVE IF NEEDED
drive.flush_and_unmount()

In [ ]:
#@title LOADING NECESSARY FILES { form-width: "500px" }
#@markdown Load all the .xvg and .dat files for graphing. All MMPBSA output was also included.
#LOAD Gromacs Files
dirpath ='drive/MyDrive/My Thesis/MD Movies/raw_files/RERUN/'
RMSD_FILES = glob.glob(os.path.join(dirpath,"*","*RMSD**xvg"))
RMSF_CALPHA_FILES = glob.glob(os.path.join(dirpath,"*","*cAlphaRMSF.xvg"))
RMSF_LIGAND_FILES = glob.glob(os.path.join(dirpath,"*","com**ligandRMSF.xvg"))
PROT_GYRATION_FILES = glob.glob(os.path.join(dirpath,"*","com**ProtOnly_gyrate**xvg"))
COMP_GYRATION_FILES = glob.glob(os.path.join(dirpath,"*","com**ProtLig_gyrate**xvg"))
SASA_FILES = [files for files in glob.glob(os.path.join(dirpath,"*","com**sasa.xvg")) if not fnmatch.fnmatch(files, "*avg_perres*")]
SASA_PERRES_FILES = [files for files in glob.glob(os.path.join(dirpath,"*","com**sasa.xvg")) if fnmatch.fnmatch(files, "*avg_perres*")]
SASA_SOLVATION_FILES = glob.glob(os.path.join(dirpath,"*","*solvation*"))
DSSP = glob.glob(os.path.join(dirpath, "*","com**sec_structure.dat"))

#LOAD MMPBSA Files
dirpath_MMPBSA='drive/MyDrive/My Thesis/MD Movies/raw_files/RERUN_MMPBSA/'
PERRES_AVG = glob.glob(os.path.join(dirpath_MMPBSA,"*","*perresavg*"))
PERRES_PLOT = glob.glob(os.path.join(dirpath_MMPBSA,"*","*perresplot*"))
PERRES_FRAMES = glob.glob(os.path.join(dirpath_MMPBSA,"*","*perresperframe*"))
E_DELTA = glob.glob(os.path.join(dirpath_MMPBSA,"*","*Delta.csv*"))
E_COMPLEX = glob.glob(os.path.join(dirpath_MMPBSA,"*","*Complex.csv*"))

#LOAD Additional Info
dirpath_add ='drive/MyDrive/My Thesis/MD Movies/raw_files/RERUN/*RMSD*'
RMSD_FILES_PROT = glob.glob(os.path.join(dirpath_add,"*proteinOnlyRMSD**xvg"))
RMSD_FILES_CALPHA = glob.glob(os.path.join(dirpath_add,"*calphaRMSD**xvg"))

In [ ]:
#@title CALCULATE THE PROTEN RMSD
ref_prot_rmsd = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','RMSD','references',"*proteinOnlyRMSD*"))

alpha_prot_rmsd = XVGReader(ref_prot_rmsd[0])
omic_prot_rmsd = XVGReader(ref_prot_rmsd[1])

fig, ax = plt.subplots(figsize=(8*(len(RMSD_FILES_PROT)),6), ncols=(len(RMSD_FILES_PROT)))
for i, cpx in enumerate(RMSD_FILES_PROT):
  file_name_template = os.path.basename(cpx).split("_")

  complex_file = XVGReader(cpx)

  ax[i].plot([ts.data[0]/1000 for ts in alpha_prot_rmsd], [ts.data[1] for ts in alpha_prot_rmsd], label = 'alpha RMSD', color = 'green')
  ax[i].plot([ts.data[0]/1000 for ts in omic_prot_rmsd], [ts.data[1] for ts in omic_prot_rmsd], label = 'omic RMSD', color = 'red')
  ax[i].plot([ts.data[0]/1000 for ts in complex_file], [ts.data[1] for ts in complex_file], label = f'{"_".join(file_name_template[0:3])} RMSD', color = 'blue')
  ax[i].set_title(f"RMSD Overlay (protein): alphWT vs omicWT vs {file_name_template[2]}_mut")
  ax[i].set_xlabel("time (ns)")
  ax[i].set_ylabel("RMSD (A)")
  ax[i].set_ylim(0, 0.6)
  ax[i].legend()

plt.show()


In [ ]:
#@title CALCULATE THE LIGAND RMSD
ref_rmsd = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','RMSD','references',"*ligandRMSD*"))
alpha_rmsd = XVGReader(ref_rmsd[0])
omic_rmsd = XVGReader(ref_rmsd[1])
comps = glob.glob(os.path.join(dirpath,"*","com**ligandRMSD**xvg"))


fig, ax = plt.subplots(figsize=(8*(len(comps)),6), ncols=(len(comps)))
for i, cpx in enumerate(comps):
  file_name_template = os.path.basename(cpx).split("_")

  complex_file = XVGReader(cpx)

  ax[i].plot([ts.data[0] for ts in alpha_rmsd], [ts.data[1] for ts in alpha_rmsd], label = 'alpha RMSD', color = 'green')
  ax[i].plot([ts.data[0] for ts in omic_rmsd], [ts.data[1] for ts in omic_rmsd], label = 'omic RMSD', color = 'red')
  ax[i].plot([ts.data[0]/1000 for ts in complex_file], [ts.data[1] for ts in complex_file], label = f'{"_".join(file_name_template[0:3])} RMSD', color = 'blue')
  ax[i].set_title(f"RMSD Overlay (ligand): alph vs omic vs {file_name_template[2]}_mut")
  ax[i].set_xlabel("time (ns)")
  ax[i].set_ylabel("RMSD (A)")
  ax[i].set_ylim(0, 35)
  ax[i].legend()

plt.show()


In [ ]:
#@title CALCULATE THE C-ALPHA RMSD
ref_calpha_rmsd = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','RMSD','references',"*calpha*"))

alpha_calpha_rmsd = XVGReader(ref_calpha_rmsd[0])
omic_calpha_rmsd = XVGReader(ref_calpha_rmsd[1])

fig, ax = plt.subplots(figsize=(8*(len(RMSD_FILES_CALPHA)),6), ncols=(len(RMSD_FILES_CALPHA)))
for i, cpx in enumerate(RMSD_FILES_CALPHA):
  file_name_template = os.path.basename(cpx).split("_")

  complex_file = XVGReader(cpx)

  ax[i].plot([ts.data[0]/1000 for ts in alpha_calpha_rmsd], [ts.data[1] for ts in alpha_calpha_rmsd], label = 'alpha RMSD', color = 'green')
  ax[i].plot([ts.data[0]/1000 for ts in omic_calpha_rmsd], [ts.data[1] for ts in omic_calpha_rmsd], label = 'omic RMSD', color = 'red')
  ax[i].plot([ts.data[0]/1000 for ts in complex_file], [ts.data[1] for ts in complex_file], label = f'{"_".join(file_name_template[0:3])} RMSD', color = 'blue')
  ax[i].set_title(f"RMSD Overlay (C-alpha): alph vs omic vs {file_name_template[2]}_mut")
  ax[i].set_xlabel("time (ns)")
  ax[i].set_ylabel("RMSD (A)")
  ax[i].set_ylim(0, 0.5)
  ax[i].legend()

plt.show()

In [ ]:
#@title PROTEIN-LIGAND GYRATION
ref_gyrate = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','Gyration Data','references',"*ProtLig*"))

alpha_gyr = XVGReader(ref_gyrate[0])
omic_gyr = XVGReader(ref_gyrate[1])


fig, ax = plt.subplots(figsize=(len(COMP_GYRATION_FILES)*6,6), ncols=len(COMP_GYRATION_FILES))
for index, files in enumerate(COMP_GYRATION_FILES):
  file_name_template = os.path.basename(files).split("_")

  comp = XVGReader(files)
  ax[index].plot([ts.data[0]/1000 for ts in alpha_gyr], [ts.data[1] for ts in alpha_gyr], label = 'alpha gyr', color = 'green', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0]/1000 for ts in omic_gyr], [ts.data[1] for ts in omic_gyr], label = 'omic gyr', color = 'red', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0]/1000 for ts in comp], [ts.data[1] for ts in comp], label = f'{"_".join(file_name_template[0:3])} gyr', color = 'blue')
  ax[index].set_title(f"Radius of gyration (total and around axes) {file_name_template[1]}")
  ax[index].set_xlabel("Time (ns)")
  ax[index].set_ylabel("Rg")
  ax[index].set_ylim(0,6)
  ax[index].legend()

plt.show()

In [ ]:
#@title PROTEIN ONLY GYRATION
ref_prot_gyrate = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','Gyration Data','references',"*ProtOnly*"))

alpha_prot_gyr = XVGReader(ref_prot_gyrate[0])
omic_prot_gyr = XVGReader(ref_prot_gyrate[1])

fig, ax = plt.subplots(figsize=(len(PROT_GYRATION_FILES)*6,6), ncols=len(PROT_GYRATION_FILES))
for index, files in enumerate(PROT_GYRATION_FILES):
  file_name_template = os.path.basename(files).split("_")

  comp = XVGReader(files)
  ax[index].plot([ts.data[0]/1000 for ts in alpha_prot_gyr], [ts.data[1] for ts in alpha_prot_gyr], label = 'alpha gyr', color = 'green', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0]/1000 for ts in omic_prot_gyr], [ts.data[1] for ts in omic_prot_gyr], label = 'omic gyr', color = 'red', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0]/1000 for ts in comp], [ts.data[1] for ts in comp], label = f'{"_".join(file_name_template[0:3])}_mut gyr', color = 'blue')
  ax[index].set_title(f"Radius of gyration PROTEIN_ONLY (total and around axes) {file_name_template[1]}", fontsize=8)
  ax[index].set_xlabel("Time (ns)")
  ax[index].set_ylabel("Rg")
  ax[index].set_ylim(1.75,2.1)
  ax[index].legend()

plt.show()

In [ ]:
#@title PROTEIN ONLY GYRATION
ref_rmsf = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','RMSF','c_alpha',"*WT*"))

alpha_ca_rmsf = XVGReader(ref_rmsf[0])
omic_ca_rmsf = XVGReader(ref_rmsf[1])

fig, ax = plt.subplots(figsize=(18,4*len(RMSF_CALPHA_FILES)), ncols=len(ref_rmsf), nrows=len(RMSF_CALPHA_FILES))

for i, files in enumerate(RMSF_CALPHA_FILES):

  file_name_template = os.path.basename(files).split("_")
  comp = XVGReader(files)
  ax[i,0].plot([ts.data[0] for ts in alpha_ca_rmsf][:-2], [ts.data[1] for ts in alpha_ca_rmsf][:-2], label = 'alpha rmsf', color = 'green', linewidth=0.5)
  ax[i,0].plot([ts.data[0] for ts in comp][:-2], [ts.data[1] for ts in comp][:-2], label = f'{"_".join(file_name_template[0:3])}_mut rmsf', color = 'blue', linewidth=0.5)
  ax[i,0].set_title(f"Alpha Carbon RMSF: Alpha Var vs {file_name_template[2]} mutant")
  ax[i,0].bar([ts.data[0] for ts in alpha_ca_rmsf][:-2], np.array([ts.data[1] for ts in alpha_ca_rmsf][:-2]) - np.array([ts.data[1] for ts in comp][:-2]),
              label = "Euclidean Distance (RMSF Stat Diff)", color='orange')
  ax[i,0].set_xlabel("RES ID")
  ax[i,0].set_ylabel("RMSF (A)")
  ax[i,0].set_ylim(-.25,1.2)
  ax[i,0].legend(fontsize=6)

  ax[i,1].plot([ts.data[0] for ts in omic_ca_rmsf][:-2], [ts.data[1] for ts in omic_ca_rmsf][:-2], label = 'omic rmsf', color = 'red', linewidth=0.5)
  ax[i,1].plot([ts.data[0] for ts in comp][:-2], [ts.data[1] for ts in comp][:-2], label = f'{"_".join(file_name_template[0:3])} rmsf', color = 'blue', linewidth=0.5)
  ax[i,1].set_title(f"Alpha Carbon RMSF: Omic Var vs {file_name_template[2]} mutant")
  ax[i,1].set_xlabel("RES ID")
  ax[i,1].set_ylabel("RMSF (A)")
  ax[i,1].bar([ts.data[0] for ts in omic_ca_rmsf],np.array([ts.data[1] for ts in omic_ca_rmsf]) - np.array([ts.data[1] for ts in comp][:-2]),
            label = "Euclidean Distance (RMSF Stat Diff)", color='orange')
  ax[i,1].set_ylim(-.25,1.2)
  ax[i,1].legend(fontsize=6)

plt.subplots_adjust(wspace=0.2, hspace=len(RMSF_CALPHA_FILES)*0.1)
plt.show()

In [ ]:
#@title PROTEIN RBM GYRATION
ref_rmsf = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','RMSF','RBM',"*WT*"))

alpha_rbm_rmsf = XVGReader(ref_rmsf[0])
omic_rbm_rmsf = XVGReader(ref_rmsf[1])

fig, ax = plt.subplots(figsize=(14,4*len(RMSF_LIGAND_FILES)), ncols=len(ref_rmsf), nrows=len(RMSF_LIGAND_FILES))

for i, files in enumerate(RMSF_LIGAND_FILES):

  file_name_template = os.path.basename(files).split("_")
  comp = XVGReader(files)
  ax[i,0].plot([ts.data[0] for ts in alpha_rbm_rmsf], [ts.data[1] for ts in alpha_rbm_rmsf], label = 'alpha rmsf', color = 'green', linewidth=0.5)
  ax[i,0].plot([ts.data[0] for ts in comp], [ts.data[1] for ts in comp], label = f'{"_".join(file_name_template[0:3])}_mut rmsf', color = 'blue', linewidth=0.5)
  ax[i,0].bar([ts.data[0] for ts in alpha_rbm_rmsf], np.array([ts.data[1] for ts in alpha_rbm_rmsf]) - np.array([ts.data[1] for ts in comp][2:]),
              label = "Euclidean Distance (RMSF Stat Diff)", color='orange')
  ax[i,0].set_title(f"RBM RMSF: Alpha Var vs {file_name_template[2]} mutant")
  ax[i,0].set_xlabel("RES ID")
  ax[i,0].set_ylabel("RMSF (A)")
  ax[i,0].set_ylim(-0.25,0.65)
  ax[i,0].legend(fontsize=8)

  print(f'average for alpha vs {file_name_template[2]}: {np.average(np.array([ts.data[1] for ts in alpha_rbm_rmsf]) - np.array([ts.data[1] for ts in comp][2:]))}')

  ax[i,1].plot([ts.data[0] for ts in omic_rbm_rmsf], [ts.data[1] for ts in omic_rbm_rmsf], label = 'omic rmsf', color = 'red', linewidth=0.5)
  ax[i,1].plot([ts.data[0] for ts in comp], [ts.data[1] for ts in comp], label = f'{"_".join(file_name_template[0:3])}_mut rmsf', color = 'blue', linewidth=0.5)
  ax[i,1].set_title(f"RBM RMSF: Omic Var vs {file_name_template[2]} mutant")
  ax[i,1].bar([ts.data[0] for ts in omic_rbm_rmsf], np.array([ts.data[1] for ts in omic_rbm_rmsf]) - np.array([ts.data[1] for ts in comp][2:]),
              label = "Euclidean Distance (RMSF Stat Diff)", color='orange')
  ax[i,1].set_xlabel("RES ID")
  ax[i,1].set_ylabel("RMSF (A)")
  ax[i,1].set_ylim(-0.25,0.65)
  ax[i,1].legend(fontsize=8)

  print(f'average for omic vs {file_name_template[2]}: {np.average(np.array([ts.data[1] for ts in omic_rbm_rmsf]) - np.array([ts.data[1] for ts in comp][2:]))}')

plt.subplots_adjust(wspace=0.2, hspace=len(RMSF_CALPHA_FILES)*0.1)
plt.show()

In [ ]:
#@title SOLVENT ACCESSIBLE SURFACE AREA
ref_sasa = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','SASA','sasa_files',"*complex_sasa.xvg"))

alpha_sasa = XVGReader(ref_sasa[1])
omic_sasa = XVGReader(ref_sasa[0])


fig, ax = plt.subplots(figsize=(len(SASA_FILES)*6,6), ncols=len(SASA_FILES))
for index, files in enumerate(SASA_FILES):
  file_name_template = os.path.basename(files).split("_")

  comp = XVGReader(files)

  ax[index].plot([ts.data[0]/1000 for ts in alpha_sasa], [ts.data[1] for ts in alpha_sasa], label = 'alpha gyr', color = 'green', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0]/1000 for ts in omic_sasa], [ts.data[1] for ts in omic_sasa], label = 'omic gyr', color = 'red', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0]/1000 for ts in comp], [ts.data[1] for ts in comp], label = f'{file_name_template[0]}_{file_name_template[2]}_mut', color = 'blue')
  ax[index].set_title(f"Solvent Accessible Surface {file_name_template[1]}")
  ax[index].set_xlabel("Time (ns)")
  ax[index].set_ylabel(r"Area (nm\S2\N)")
  ax[index].set_ylim(90,140)
  ax[index].legend()

plt.show()

In [ ]:
#@title AVG PER RESIDUE SOLVENT ACCESSIBLE SURFACE AREA
ref_avg_perres_sasa = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','SASA','avg_perres_sasa',"*complex_avg_perres_sasa.xvg"))

alpha_avgpr_sasa = XVGReader(ref_avg_perres_sasa[1])
omic_avgpr_sasa = XVGReader(ref_avg_perres_sasa[0])


fig, ax = plt.subplots(figsize=(16,len(SASA_PERRES_FILES)*3), nrows=len(SASA_PERRES_FILES))
for index, files in enumerate(SASA_PERRES_FILES):
  file_name_template = os.path.basename(files).split("_")

  comp = XVGReader(files)
  ax[index].plot([ts.data[0] for ts in omic_avgpr_sasa], [ts.data[1] for ts in omic_avgpr_sasa], label = 'omic gyr', color = 'red', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0] for ts in alpha_avgpr_sasa], [ts.data[1] for ts in alpha_avgpr_sasa], label = 'alpha gyr', color = 'green', linestyle="--", linewidth='0.5')
  ax[index].plot([ts.data[0] for ts in comp], [ts.data[1] for ts in comp], label = f'{file_name_template[0]}_{file_name_template[2]}_mut', color = 'blue')

  ax[index].set_title(f"Solvent Accessible Surface Area: Per Residue AVG {file_name_template[1]}")
  ax[index].set_xlabel("ResID")
  ax[index].set_ylabel(r"AVG Area (nm\S2\N)", fontsize=8)
  ax[index].set_xlim(436,506)
  ax[index].set_ylim(0,3)
  ax[index].legend(fontsize=8)

plt.subplots_adjust(hspace=len(SASA_PERRES_FILES)*0.3)
plt.show()

In [ ]:
#@title DSSP Parsing and Visualization for Protein Secondary Structure
#CUSTOM ALGO FOR DSSP Parsing
dssp_dir ='drive/MyDrive/My Thesis/MD Movies/raw_files/DSSP'
all_dat = glob.glob(os.path.join(dssp_dir,"*complex**.dat")) + DSSP

custom_colors = ['#E66100', '#5D3A9B', '#009E73']
cmap = mcolors.ListedColormap(custom_colors)

for dssp_file in all_dat:
  fname = os.path.basename(dssp_file).split("_")
  with open(dssp_file,"r") as rd:
    s = rd.readlines()
  #parser mapping index
    secondary_structure_map = {'S':1, '~': 2, 'P':3}
  #S for SHEET, ~ for coil. P for turn


    cont = [] ; frame_count = 0
    for frame in s:
      frame_map = [secondary_structure_map[x] if x in secondary_structure_map else 0 for i,x in enumerate(frame.strip())]
      if len(frame_map) == 192 or len(frame_map) == 194:
        if frame_count==0:
          cont.append(frame_map[:192])
        elif (frame_count%3) + 1 == 2:
          cont.append(frame_map[:192])
        #enable line if want to probe whole frame set and disable nested if else
        # cont.append(frame_map)
        frame_count += 1
    rd.close()
  matrix = np.array(cont)
  print(matrix.shape)
  dssp = np.rot90(np.array(cont),k=3, axes=(1,0))
  fig, ax = plt.subplots(figsize=(18,len(all_dat)*0.75))
  ax.set_title(f"DSSP conformational map (interval3frames) {fname[0]}_{fname[1]}")
  # ax.set_yticks(list(range(333,527)))
  ax.set_yticks(np.linspace(0,192,10))
  ax.set_yticklabels(np.linspace(524,333,10, dtype="int64"))

  ax.set_xticks(np.linspace(1,335,8))
  ax.set_xticklabels(np.linspace(0,1000,8, dtype='int64'))

  img = ax.imshow(dssp, cmap = cmap, aspect="auto")


  cbar = fig.colorbar(img, ax=ax)
  cbar.ax.set_title('legend')
  cbar.ax.set_yticks([1.30,2,2.6],["sheet","coil","turn"])
plt.show()

In [ ]:
#@title DSSP Parsing and Visualization for Protein Secondary Structure (RBM Only)
#CUSTOM ALGO FOR DSSP Parsing
dssp_dir ='drive/MyDrive/My Thesis/MD Movies/raw_files/DSSP'
all_dat = glob.glob(os.path.join(dssp_dir,"*complex**.dat")) + DSSP

custom_colors = ['#E66100', '#5D3A9B', '#009E73']
cmap = mcolors.ListedColormap(custom_colors)

#Others
ref_rbm_rmsf = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files','RMSF','RBM',"*WT*"))
all_dat_rbm = ref_rbm_rmsf + RMSF_LIGAND_FILES

#Fixing the sequence of samples coz everything should match
all_dat_rbm[0], all_dat_rbm[1] = all_dat_rbm[1], all_dat_rbm[0]

#colormaps
cmap_barsticks = plt.cm.get_cmap('hsv_r')

for rbm, dssp_file in zip(all_dat_rbm,all_dat):
  fname = os.path.basename(dssp_file).split("_")
  with open(dssp_file,"r") as rd:
    s = rd.readlines()
  #parser mapping index
    secondary_structure_map = {'S':1, '~': 2, 'P':3}
  #S for SHEET, ~ for coil. P for turn


    cont = [] ; frame_count = 0
    for frame in s:
      frame_map = [secondary_structure_map[x] if x in secondary_structure_map else 0 for i,x in enumerate(frame.strip())]
      if len(frame_map) == 192 or len(frame_map) == 194:
        if frame_count==0:
          cont.append(frame_map[436-333:506-333])
        elif (frame_count%3) + 1 == 2:
          cont.append(frame_map[436-333:506-333])
        #enable line if want to probe whole frame set and disable nested if else
        # cont.append(frame_map)
        frame_count += 1
    rd.close()
  matrix = np.array(cont)
  print(matrix.shape)
  dssp = np.rot90(np.array(cont),k=3, axes=(1,0))
  fig, ax = plt.subplots(figsize=(14,len(all_dat)*0.75), ncols=2, width_ratios=[4,1])
  plt.subplots_adjust(hspace=len(RMSF_LIGAND_FILES)*0,wspace=0)
  ax[0].set_title(f"DSSP conformational map (interval3frames) {fname[0]}_{fname[1]}")
  # ax.set_yticks(list(range(333,527)))
  ax[0].set_yticks(np.linspace(0,(506-333)-(436-333),10))
  ax[0].set_yticklabels(np.linspace(506,436,10, dtype="int64"))
  ax[0].set_xticks(np.linspace(1,335,8))
  ax[0].set_xticklabels(np.linspace(0,1000,8, dtype='int64'))
  img = ax[0].imshow(dssp, cmap = cmap, aspect="auto")


  file_name_template = os.path.basename(rbm).split("_")
  comp = XVGReader(rbm)

  if len([ts.data[0] for ts in comp]) >=69:
    ax[1].barh([ts.data[0] for ts in comp][2:], [ts.data[1] for ts in comp][2:], color=cmap_barsticks([ts.data[1] for ts in comp][2:]),label = f'{"_".join(file_name_template[0:3])}_mut rmsf', linewidth=0.5)
  else:
    ax[1].barh([ts.data[0] for ts in comp], [ts.data[1] for ts in comp], label = f'{"_".join(file_name_template[0:3])}_mut rmsf', color = 'blue', linewidth=0.5)
  ax[1].set_yticks(np.linspace(437,506,10, dtype="int64"))
  ax[1].tick_params(axis='y', labelleft=False, labelright=False, left=False)
  ax[1].set_xlim(0.05,0.6)
  ax[1].set_ylim(438, 506)
  ax[1].set_ylabel('Per-Residue Fluctuation of RBM Residues', rotation=-90, labelpad=15)

# Adjust the position of the ylabel
  plt.gca().yaxis.set_label_coords(1.05, 0.5)
  # ax[1].set_yticklabels(np.linspace(506,436,10, dtype="int64"))
  # ax[1].set_xticks(np.linspace(1,335,8))
  # ax[1].set_xticklabels(np.linspace(0,1000,8, dtype='int64'))


  # cbar = fig.colorbar(img, ax=ax[0])
  # cbar.ax.set_title('legend')
  # cbar.ax.set_yticks([1.30,2,2.6],["sheet","coil","turn"])


plt.show()

In [ ]:
#@title 3d PCA Plot
pca_samples = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files/RERUN/df_pca','com*'))
pca_refs = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files/RERUN/df_pca','*WT*'))

alpha_pca = pd.read_csv(pca_refs[0])
omic_pca = pd.read_csv(pca_refs[1])

for files in pca_samples:
    try:
      print(f'Processing {files}')
      df_read = pd.read_csv(files, index_col=0)
      df=df_read[:576]    #only consider 333-524 res (Lan et al., 2021)

      sns.set_style("darkgrid")
      fig,ax = plt.subplots(figsize=(12,6), ncols=2)

      fname = os.path.basename(files)
      sns.kdeplot(x = df.PC1, y = df.PC2, cmap='magma_r', label = f"{fname} PCA", ax=ax[0])
      sns.kdeplot(x = alpha_pca.PC1, y = alpha_pca.PC2, fill=True, label="alpha_WT PCA", cmap='magma_r', ax=ax[0])

      sns.kdeplot(x = df.PC1, y = df.PC2, cmap='magma_r', label = f"{fname} PCA", ax=ax[1])
      # sns.kdeplot(x = alpha_pca.PC1, y = alpha_pca.PC2, fill=True, cmap='magma_r', ax=ax[1])
      sns.kdeplot(x = omic_pca.PC1, y = omic_pca.PC2, fill=True, cmap='magma_r', label="omic_WT PCA", ax=ax[1])

      ax[0].set_title(f"PCA Plot visualization of {fname.removesuffix('.csv').removeprefix('com')[3:]} and Alpha_WT")
      ax[0].set_xlim(-60,90)
      ax[0].set_ylim(-60,65)
      ax[1].set_title(f"PCA Plot visualization of {fname.removesuffix('.csv').removeprefix('com')[3:]} and Omic_WT")
      ax[1].set_xlim(-60,90)
      ax[1].set_ylim(-60,65)
      plt.show()
    except IndexError:
        print("Error occured")

In [ ]:
#@title 3d PCA Plot (RBM Only)
pca_samples = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files/RERUN/df_pca','com*'))
pca_refs = glob.glob(os.path.join('drive/MyDrive/My Thesis/MD Movies/raw_files/RERUN/df_pca','*WT*'))

alpha_pca = pd.read_csv(pca_refs[0])
omic_pca = pd.read_csv(pca_refs[1])

for files in pca_samples:
    try:
      print(f'Processing {files}')
      df_read = pd.read_csv(files, index_col=0)
      df=df_read[309:522]    #only consider 333-524 res (Lan et al., 2021)

      sns.set_style("darkgrid")
      fig,ax = plt.subplots(figsize=(12,6), ncols=2)

      fname = os.path.basename(files)
      sns.kdeplot(x = df.PC1, y = df.PC2, cmap='magma_r', ax=ax[0])
      sns.kdeplot(x = alpha_pca.PC1[309:522], y = alpha_pca.PC2[309:522], fill=True, cmap='magma_r', ax=ax[0])

      sns.kdeplot(x = df.PC1, y = df.PC2, cmap='magma_r', ax=ax[1])
      # sns.kdeplot(x = alpha_pca.PC1, y = alpha_pca.PC2, fill=True, cmap='magma_r', ax=ax[1])
      sns.kdeplot(x = omic_pca.PC1[309:522], y = omic_pca.PC2[309:522], fill=True, cmap='magma_r', ax=ax[1])
      ax[0].set_title(f"PCA Plot visualization of {fname.removesuffix('.csv').removeprefix('com')[3:]} and Alpha_WT (RBM ED)")
      ax[0].set_xlim(-60,90)
      ax[0].set_ylim(-60,65)
      ax[1].set_title(f"PCA Plot visualization of {fname.removesuffix('.csv').removeprefix('com')[3:]} and Omic_WT (RBM ED)")
      ax[1].set_xlim(-60,90)
      ax[1].set_ylim(-60,65)
      plt.show()
    except IndexError:
        print("Error occured")